In [ ]:
import tensorflow as tf
import random

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

#파라미터 값
learning_rate = 0.001
training_epochs = 5
batch_size =100

#input placeholder
X = tf.placeholder(tf.float32, [None, 784]) #784개의 값을 가져오고 N개의 이미지
X_img = tf.reshape(X,[-1,28,28,1]) 
Y = tf.placeholder(tf.float32, [None, 10])

#Conv layer1  Imgin shape=(?,28,28,1)
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01)) #32개의 filer사용

   # Conv layer  --> (?,28,28,32) 32개의 필터를 사용해서 32개의 Conv값이 생긴다.
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1],
                   strides=[1,2,2,1], padding='SAME') #relu를 통과시키고 max_pooling을 한다.
   # kernal size는 2*2 stride는 2*2 
   # Pooling Max를 거치면 stride가 2이므로  14*14 --> (?,14,14,32) 32개의 channel


#Conv layer 2 Imgin shape=(?,14,14,32) 위의 Conv layer1 거친 후이므로.
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01)) #32는 앞에서 받은 값.이번엔 64개의 필터
    #Conv layer  --> (?,14,14,64)
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2) # relu를 통화. 사이즈 변경 안하고 pooling한다.
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    #pooling 거치면서 stride가 2이므로 7*7이되다.--> (?,7,7,64)
L2_flat = tf.reshape(L2, [-1,7*7*64]) # -1은 n개의 값이고, 쫙 펼치기 위해 reshape함. 7*7*64가 n개.

# 마지막 단계 Fully Connected layer
    #final FC 7*7*64 inputs ==> 10 outputs 0~9까지 예측하기위해.
W3 = tf.get_variable("W3", shape=[7*7*64,10],
                        initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2_flat, W3) + b # xw + b : x는 L2

#define cost/loss & optimizer 이전과 같이 하면 된다
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

#initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

